In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

Load Dataset

In [ ]:
data_path = "./data/UL_ECE_IoT_2025.csv"

df = pd.read_csv(data_path)

print("Dataset shape:", df.shape)
df.head()

Basic Dataset Inspection

In [ ]:
# Column names
print("Columns:\n", df.columns.tolist())

# Data types
df.info()

Identify Label Column

In [ ]:
possible_labels = [col for col in df.columns if 
                   col.lower() in ["label", "class", "attack", "target"]]

print("Possible label columns:", possible_labels)

Label Distribution 

In [ ]:
label_col = possible_labels[0]  # change if needed

df[label_col].value_counts()


Check Missing & Duplicate Values

In [ ]:
print("Missing values:\n", df.isnull().sum())

print("Duplicate rows:", df.duplicated().sum())
